In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Library/Packages

* Keras -> Model/Prediction/Layers
* numpy -> Hesaplamalar ve pandas'a yardımcı olması için 
* pandas -> Data için
* matplotlib -> Görselleştirmek için

In [ ]:
from keras.layers import Conv2D, LeakyReLU, Dense, Flatten, Dropout, MaxPool2D # Layers
from keras.models import Sequential # Sequential Model
from keras.optimizers import Adam,RMSprop # optimizer
from keras.preprocessing.image import ImageDataGenerator # data generator
from keras.callbacks import ReduceLROnPlateau 

# Loading Dataset

#### MNIST dataset:

* 60000 tane el yazması 0-9'a kadar sayılar bulunmaktadır.
* Her fotoğraf 28*28 px'den oluşmaktadır ve gri renktedir.
* Her pixel 0-255 arasında bir değer almaktadır. '0' siyahı, '255' beyazı temsil etmektedir.
* Her bir sayının label'i 0-9 olarak tanımlanmıştır.
* Data 758 sütun'dan oluşmaktadır. İlk sütun label olarak tanımlanmıştır, diğer 784 tane sütun ise pixellerden oluşmaktadır.

In [ ]:
train = pd.read_csv("../input/mnist-in-csv/mnist_train.csv")
test = pd.read_csv("../input/mnist-in-csv/mnist_test.csv")
print("\n",train.info())
print(test.info())

In [ ]:
train.head()

In [ ]:
print(train.shape)
train.describe()

# Prepare Train and Test 

In [ ]:
x_train = train.drop(['label'], axis=1) # x_train'e label dışındaki tüm px değerlerimi alıyorum
y_train = train.label # label değeri yani sayının değerini 
x_test = test.drop(['label'], axis=1) # x_train ile aynı şekilde 
y_test = test.label # y_train ile aynı şekilde 

In [ ]:
print("x_train shape before reshape:", x_train.shape)
print("x_test shape before reshape:", x_test.shape)
print("y_train shape before reshape:", y_train.shape)
print("y_test shape before reshape:", y_test.shape)

x_train = np.array(x_train).reshape(-1, 28, 28, 1)
x_test = np.array(x_test).reshape(-1, 28, 28, 1)

In [ ]:
from keras.utils.np_utils import to_categorical # one-hot-encoding'a çevirmek için 
y_train = to_categorical(y_train, num_classes = 10) # label encoding 
y_test = to_categorical(y_test, num_classes = 10)

##### CNN ile train etmek için np.array'e atıp reshape etmemiz gerekiyor. Yukarıda yaptığımız reshape işlemi 1 boyutlu olan pixellerimizi 3 boyut'a taşıyor. Array'imizde 60000/10000 olarak belirtilen kaç tane veri olduğunu tutuyor. Diğer kısım ise 3 boyutlu input'u gösteriyor. Bizim verimiz 28x28x1'lik 3 boyutlu 60000/10000 tane inputtan oluşuyor. 
* Kısaca (sample_size, 28x28x1)

In [ ]:
print("x_train shape after reshape:", x_train.shape)
print("x_test shape after reshape:", x_test.shape)
print("y_train shape after reshape:", y_train.shape)
print("y_test shape after reshape:", y_test.shape)

# Normalization Data
* Datamızı 0-1 arasına scale ediyoruz

In [ ]:
x_train = x_train/255.0
x_test = x_test/255.0

# Building Model

###### Model oluşturmanın iki yolu var,
    1.  Sequential Model ile,
    2.  Function API kullanarak
* Functional API daha komplike daha zorlu modellerde ve çok output var ise kullanılabilir. Basit düzeyde model kullanmak istiyoruz. 
* Sequential'da modelimize istediğimiz şekilde sırasıyla layer ekleyebiliyoruz.

###### Modelimiz ,
* 2 tane Conv. Blok'tan oluşacak.
* Her blokt 2 tane Conv2D Layer'dan oluşacak ve aktivasyon fonksiyonu olarak LeakyRelu kullanacağız. Sonra MaxPool2D layerimiz ve son olarak Dropout yapacağız. 
* Sonrasında Flatten Layer'ımız bulunacak ardından outputmuzun olduğu Dense Layerlarımızı koyacağız

* MaxPool2D layer'ı image'in boyutunu reduce etmemize yarıyoru. Pool size(2,2) default olarak uygulandığında (28,28) olan image'imiz (14,14)' düşüyor. Bir nevi feature reducing.

* Dropout layer ise node'ları random şekilde sizin verdiğiniz oranda kapatıyor. Regularization layer'ı, over-fit engellemek için kullanıyoruz. 

* Output layer'ımız ise 10 node'dan oluşacak, sigmoid func kullanacağız output(y_train) 10 tane outputtan oluşuyor.

In [ ]:
epochs = 30
batch_size = 250
model = Sequential()

# Block 1
model.add(Conv2D(32,3, padding  ="same", input_shape=(28,28,1)))
model.add(LeakyReLU())
model.add(Conv2D(32,3, padding  ="same"))
model.add(LeakyReLU())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#Block 2
model.add(Conv2D(64,3, padding  ="same"))
model.add(LeakyReLU())
model.add(Conv2D(64,3, padding  ="same"))
model.add(LeakyReLU())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#Flatten Block
model.add(Flatten())

#Output Block
model.add(Dense(256,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(32,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(10,activation="softmax"))
model.summary()

# Compiling Model

###### Optimizers: 

    1. SGD (Stochastic gradient descent optimizer)
    2. RMSprop
    3. Adam
    4. Adamax

##### Learning Rate

    1. Eğer learning rate çok yüksek ise loss'umuz azalırken bir anda yükselebilir. En az loss'a sahip olamayabiliriz.
    2. Eğer learning rate çok az ise learning çok yavaş olur. Düzgün bir rate değeri seçmemiz gerekiyor.
    3. lr değeri 0.001 yeterli bir değerdir. Eğer işe yaramazsa daha yüksek ya da düşük değer verilebilir.
    4. Biz adam optimizer kullanacağız.

##### Loss Functions: 

    1. binary_crossentropy: Bu loss fonksiyonu daha çok tek node'lu output layerlarda kullanılıyor. 0 ve 1 classificationlarında 
    2. categorical_crossentropy: Multi-class yani birden çok outputlarda kullanılyıor. 
    3. sparse_categorical_crossentropy: Yukarıdaki ile aynı aşağıda farkından bahsediliyor.
  
    
    
* Eğer target'im one-hot encoded ise categorical_crossentropy kullanabilirsin.
Örneğin:
 [1,0,0]
 [0,1,0]
 [0,0,1]

* Eğer targetim integer ise sparse_categorical_crossentropy kullanabilirsin. 
Örneğin:
1
2
3



###### Improvement Learning Rate
* ReduceLROnPlateau() keras'ta bulunan bir callback fonksiyon, bu fonksiyon aşamalı olarak accuracy'e bakarak, (eğer improvement durduysa)lr'yi düşürerek öğrenmenin daha iyi olmasını sağlıyor.


    1. monitor: bir metric'i monitor ediyor, bu bizde val_accuracy
    2. patience: ne kadar epochs süresince bir improvement olmadı, bunu belirliyoruz, örneğin 2 epochs sonrasında bir improvement olmadıysa lr düşürüyoruz. 
    3. factor: ne kadar değerde lr düşürülecek new_lr = lr * factor (0.5)
    4. min_lr: en düşük lr band'ı (0.00001)

In [ ]:
# kullanılabilir optimizer featureları
"""
sgd = keras.optimizers.SGD(lr=1e-4, momentum=0.9)
rms_prop = keras.optimizers.RMSprop(lr=1e-4)
adam = keras.optimizers.adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
adamax = keras.optimizers.Adamax(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
adadelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001) """

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#rms_prop = RMSprop(lr=0.001)
learning_rate_reduce = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.4, 
                                            min_lr=0.00001)
loss = "categorical_crossentropy"

In [ ]:
model.compile( optimizer= adam, loss=loss ,metrics=['accuracy'])

# Image Generate

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

# Training the model

In [ ]:
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size = batch_size),
                              epochs = epochs, validation_data = (x_test, y_test),
                              steps_per_epoch = x_train.shape[0] // batch_size,
                              callbacks=[learning_rate_reduce])
result = model.evaluate(x = x_train, y = y_train)


In [ ]:
print('Accuracy:', result[1])

# Performance plotting

In [ ]:
plt.plot(history.history['loss'])
plt.title("Loss Plot", fontsize = 15)
plt.xlabel("Epochs", fontsize = 12)
plt.ylabel("Loss", fontsize = 12)
plt.grid(alpha=0.3)
plt.legend(["Train", "Test"])
plt.show()

plt.plot(history.history["accuracy"])
plt.title("Accuracy Plot")
plt.xlabel("Epochs")
plt.ylabel("Accuracy", fontsize = 12)
plt.grid(alpha=0.3)
plt.legend(["Train","Test"])
plt.show()

# Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)

# Convert predictions classes to one hot vectors 
y_pred_classes = np.argmax(y_pred,axis = 1) 

# Convert validation observations to one hot vectors
y_true = np.argmax(y_test,axis = 1) 

# compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes) 

# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))

sns.heatmap(cm, annot = True, linewidths = 0.01, cmap = "Greens", linecolor = "gray", fmt = '.1f',ax = ax)

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")

plt.show()

# Submission

In [ ]:
test_y = np.argmax(model.predict(x_test),axis =1)

In [ ]:
df_submission = pd.DataFrame([test.index+1,test_y],["ImageId","Label"]).transpose()
df_submission.to_csv("submission.csv",index=False)